# Evaluating Files declared in code


This tutorial highlights how to combine Humanloop decorated files.


### 1. Setting up imports


In [1]:
import os

from dotenv import load_dotenv
from humanloop import Humanloop
import chromadb
from openai import OpenAI
import pandas as pd

from levenshtein import compare_log_and_target
from exact_match import exact_match


load_dotenv()

True

### 2. Instantiating the Humanloop client


In [ ]:
humanloop = Humanloop(api_key=os.getenv("HL_API_KEY"))

### 3. Instantiating the vector database


In [3]:
chroma = chromadb.chromadb.Client()
collection = chroma.get_or_create_collection(name="MedQA")

# init collection into which we will add documents
knowledge_base = pd.read_parquet("../../../assets/sources/textbooks.parquet")
knowledge_base = knowledge_base.sample(5, random_state=42)
collection.add(
    documents=knowledge_base["contents"].to_list(),
    ids=knowledge_base["id"].to_list(),
)

### 4. Loading the evaluation dataset


In [4]:
datapoints_df = pd.read_json("../../../assets/datapoints.jsonl", lines=True)
datapoints = [row.to_dict() for _, row in datapoints_df.iterrows()][:20]

### 5. Declare Humanloop Files via code


In [5]:
TEMPLATE = """Answer the following question factually.

Question: {question}

Options:
- {option_A}
- {option_B}
- {option_C}
- {option_D}
- {option_E}

---

Here is some retrieved information that might be helpful.
Retrieved data:
{retrieved_data}

---

Give you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.
```
<chosen option verbatim>
---
<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>
---
<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>
```
"""

In [6]:
@humanloop.tool(
    path="Evaluations SDK Demo/Retrieval Tool",
)
def retrieval_tool(question: str) -> str:
    """Retrieve most relevant document from the vector db (Chroma) for the question."""
    response = collection.query(query_texts=[question], n_results=1)
    retrieved_doc = response["documents"][0][0]
    return retrieved_doc


@humanloop.prompt(
    path="Evaluations SDK Demo/MedQA Answer",
    model="gpt-4o",
    template=TEMPLATE,
    tools=[retrieval_tool.json_schema],
)
def ask_model(
    question: str,
    option_A: str,
    option_B: str,
    option_C: str,
    option_D: str,
    option_E: str,
) -> str:
    """Ask a question and get an answer using a simple RAG pipeline"""
    openai = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    # Retrieve context
    retrieved_data = retrieval_tool(question)
    inputs = {
        "question": question,
        "option_A": option_A,
        "option_B": option_B,
        "option_C": option_C,
        "option_D": option_D,
        "option_E": option_E,
        "retrieved_data": retrieved_data,
    }

    # Populate the Prompt template
    messages = [
        {
            "role": "user",
            "content": TEMPLATE.format(**inputs),
        }
    ]

    # Call OpenAI to get response
    chat_completion = openai.chat.completions.create(
        model="gpt-4o",
        temperature=0,
        messages=messages,
    )
    return chat_completion.choices[0].message.content


@humanloop.flow(
    path="Evaluations SDK Demo/MedQA Answer Flow",
    attributes={
        "prompt": {
            "model": "gpt-4o",
            "environment": "evaluation",
        }
    },
)
def entrypoint(
    question: str,
    option_A: str,
    option_B: str,
    option_C: str,
    option_D: str,
    option_E: str,
):
    return ask_model(
        question=question,
        option_A=option_A,
        option_B=option_B,
        option_C=option_C,
        option_D=option_D,
        option_E=option_E,
    )

### 6. Evaluate the Flow


In [7]:
humanloop.evaluations.run(
    file={
        "path": "Evaluations SDK Demo/MedQA Answer Flow",
        "callable": entrypoint,
        "type": "flow",
    },
    name="MedQA Evaluation Decorators",
    dataset={
        "datapoints": datapoints,
        "path": "Evaluations SDK Demo/Dataset",
    },
    evaluators=[
        {
            "path": "Evaluations SDK Demo/Levenshtein",
            "args_type": "target_required",
            "return_type": "number",
            "callable": compare_log_and_target,
        },
        {
            "path": "Evaluations SDK Demo/Exact Match",
            "args_type": "target_required",
            "return_type": "boolean",
            "callable": exact_match,
        },
    ],
    workers=8,
)

Evaluating your flow function corresponding to `Evaluations SDK Demo/MedQA Answer Flow` on Humanloop 



Navigate to your Evaluation:
http://localhost:3000/project/fl_f1LKeioSdpBr6vUXfFMgQ/evaluations/evr_TdSSqdd2e6SH48YI6bsT4/stats

Flow Version ID: flv_GnVlk3RpLWCcFWCJN1flF
Run ID: rn_ZS3HRPc4fnGbcHeAX8YfY

Running 'MedQA Answer Flow' over the Dataset 'Dataset' using 8 workers 

⏳ Evaluation Progress
Total Logs: 17
Total Judgments: 30



⏳ Evaluation Progress
Total Logs: 17
Total Judgments: 30



⏳ Evaluation Progress
Total Logs: 19
Total Judgments: 37



⏳ Evaluation Progress
Total Logs: 20
Total Judgments: 40



📊 Evaluation Results for Evaluations SDK Demo/MedQA Answer Flow 
+----------------------------------+---------------------+
|                                  |        Latest       |
+----------------------------------+---------------------+
|                           Run ID |        ZS3HR        |
+----------------------------------+---------------------+
|               

IS_EVALUATED {'path': 'Evaluations SDK Demo/MedQA Answer Flow', 'flow': {'attributes': {'prompt': {'environment': 'evaluation', 'model': 'gpt-4o'}}}, 'output': "```\nCommon iliac artery aneurysm\n---\nThe patient's symptoms and ultrasound findings suggest a blockage or compression in the urinary tract, leading to dilation of the right ureter and renal pelvis. A common iliac artery aneurysm can compress the ureter, causing this dilation and resulting in flank pain. \n\n- Renal artery stenosis typically causes high blood pressure and kidney dysfunction, not ureteral dilation.\n- Benign prostatic hyperplasia usually causes lower urinary tract symptoms, not unilateral ureteral dilation.\n- Diabetic nephropathy affects kidney function but does not cause ureteral dilation.\n- Urethral stricture would cause difficulty urinating but not unilateral ureteral dilation.\n\n---\nNo relevant information from the retrieved data directly addresses the medical scenario or options provided.\n```", 'inpu

[##--------------------------------------] 1/20 (5.00%) | ETA: 0s

IS_EVALUATED {'path': 'Evaluations SDK Demo/MedQA Answer', 'prompt': {'temperature': 0, 'provider': 'openai', 'endpoint': 'chat', 'model': 'gpt-4o', 'template': 'Answer the following question factually.\n\nQuestion: {question}\n\nOptions:\n- {option_A}\n- {option_B}\n- {option_C}\n- {option_D}\n- {option_E}\n\n---\n\nHere is some retrieved information that might be helpful.\nRetrieved data:\n{retrieved_data}\n\n---\n\nGive you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.\n```\n<chosen option verbatim>\n---\n<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>\n---\n<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>\n```\n', 'tools': [{'name': 'retrieval_tool', 'description': 'Retrieve most relevant document from the vector db (Chroma) for the question.', 'parameters': {'t

[####------------------------------------] 2/20 (10.00%) | ETA: 18s

...

[######################################--] 19/20 (95.00%) | ETA: 1s

IS_EVALUATED {'path': 'Evaluations SDK Demo/MedQA Answer', 'prompt': {'temperature': 0, 'provider': 'openai', 'endpoint': 'chat', 'model': 'gpt-4o', 'template': 'Answer the following question factually.\n\nQuestion: {question}\n\nOptions:\n- {option_A}\n- {option_B}\n- {option_C}\n- {option_D}\n- {option_E}\n\n---\n\nHere is some retrieved information that might be helpful.\nRetrieved data:\n{retrieved_data}\n\n---\n\nGive you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.\n```\n<chosen option verbatim>\n---\n<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>\n---\n<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>\n```\n', 'tools': [{'name': 'retrieval_tool', 'description': 'Retrieve most relevant document from the vector db (Chroma) for the question.', 'parameters': {'t

[########################################] 20/20 (100.00%) | DONE


IS_EVALUATED {'path': 'Evaluations SDK Demo/MedQA Answer', 'prompt': {'temperature': 0, 'provider': 'openai', 'endpoint': 'chat', 'model': 'gpt-4o', 'template': 'Answer the following question factually.\n\nQuestion: {question}\n\nOptions:\n- {option_A}\n- {option_B}\n- {option_C}\n- {option_D}\n- {option_E}\n\n---\n\nHere is some retrieved information that might be helpful.\nRetrieved data:\n{retrieved_data}\n\n---\n\nGive you answer in 3 sections using the following format. Do not include the quotes or the brackets. Do include the "---" separators.\n```\n<chosen option verbatim>\n---\n<clear explanation of why the option is correct and why the other options are incorrect. keep it ELI5.>\n---\n<quote relevant information snippets from the retrieved data verbatim. every line here should be directly copied from the retrieved data>\n```\n', 'tools': [{'name': 'retrieval_tool', 'description': 'Retrieve most relevant document from the vector db (Chroma) for the question.', 'parameters': {'t

[]